# NEM DATA CHALLENGE
## Data Preprocessing Code 

* Rev001 Save csv's generated by each function at different subdirs
* Rev002 Outputs all files into one single subdir
* Rev005 A pandas dataframe is used in def: F2 to write to disk instead of dask.dataframe, as in previous versions. Also, F4 has been merged into F3. These changes have reduced run-time to about 5.5 hrs.

In [160]:
import os
import time
import numpy as np
import pandas as pd
import dask.dataframe as dd
from datetime import datetime

In [161]:
def main():
    '''Dataset Preprocessing Main module'''

    # call to Defs
    F1_csv_per_asset()
    F2_csv_per_sensor()
    F3_csv_resample()
    df = F4_all_in_one_csv()
    return df

In [162]:
def read_translations():
    '''Read translations.csv, which has been previously converted from the original XML format by 
    reading the file in (microsoft) PowerBI and exporting to csv'''
    
    in_path = os.getcwd()
    in_file = 'translations.csv'
    in_csv  = os.path.join(in_path,in_file)
    df_trans = pd.read_csv(in_csv)
    assets = df_trans.asset.unique()
    sensors = df_trans.sensorID.unique()
    return assets, sensors, df_trans    

In [163]:
def F1_csv_per_asset():
    '''Reads original dataset.csv file and divides it into a csv file per Asset. 
    Additionally, it translates Asset_variables into Unified Sensor IDs.'''
    
    t0 = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print("Executing F1_csv_asset at {}".format(t0))
    # FUN.CALL define in/output dirs
    out_path = files_location()
    in_path  = os.getcwd()
    # FUN.CALL read_translations
    assets, sensors, df_trans = read_translations()

    # read principal dataset
    in_file = 'dataset.csv'
    in_csv=os.path.join(in_path,in_file)
    ddf = dd.read_csv(in_csv)
    
    # translate asset variables into unified sensor ID
    ddf = ddf.merge(df_trans, how='inner', on=['asset', 'variable'])
    ddf = ddf.drop('variable', axis=1)
    
    # loop over the dataset and separate rows per asset
    for asset in assets:
        t0 = time.time()
        mask = (ddf['asset']==asset)
        ddf_asset = ddf[mask].drop(['asset'], axis=1)
        # write to disk
        out_file = 'F1-'+ asset +'-*.csv'
        out_csv=os.path.join(out_path,out_file)
        dd.to_csv(ddf_asset,out_csv, index=False)
        t1 = time.time()
        print("File {} has been written in {}".format(asset, t1-t0))
        #break #<== remove firts # for code testing, allows one loop only
    
    # FUN.CALL Remove files with no data created by dask.dataframe during dd.to_csv
    remove_empty_files(out_path)

In [164]:
def F2_csv_per_sensor():
    '''F2 reads each csv file generated by F1 and further divides them into one csv file per SensorID, 
    rendering a total of 140 individual csv files (5 assets * 28 sensorsIDs)'''

    t0 = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print("Executing F2:csv_sensor at {}".format(t0))
    # FUN.CALL
    out_path = files_location()
    in_path  = out_path # modify if in/out-paths differ 
    # FUN.CALL
    assets, sensors, df_trans = read_translations() 
        
    for asset in assets:
        in_file = 'F1-'+ asset + '-*.csv'
        in_csv=os.path.join(in_path,in_file)
        ddf = dd.read_csv(in_csv)
        
        for idx, sensor in enumerate(sensors, start=1):
            t0 = time.time()
            mask = (ddf['sensorID']==sensor)
            ddf_sensor = (ddf[mask].drop(['sensorID'], axis=1)
                                   .rename(columns={'value':sensor})
                                   .compute())
            df = pd.DataFrame(ddf_sensor)
            # write to csv file per asste&sensor
            out_file = 'F2-'+ asset + '-' + sensor + '.csv' 
            out_csv=os.path.join(out_path,out_file)
            df.to_csv(out_csv, index=False)
            t1 = time.time()
            print("File {} has been written in {}".format(asset +'-'+ str(idx), t1-t0))     

In [165]:
def F3_csv_resample():
    '''Resample each csv file to a sampling freq. of 10 min '''
    
    t0 = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print("Executing F3 csv_resample at {}".format(t0))
    #FUN.CALL
    out_path = files_location()
    in_path  = out_path # modify if in/out-paths differ
    # FUN.CALL
    assets, sensors, df_trans = read_translations() 
       
    for asset in assets:
        ta0 = time.time()
        for idx, sensor in enumerate(sensors, start=1):
            ts0 = time.time()
            in_file = 'F2-'+ asset + '-' + sensor + '.csv' 
            in_csv=os.path.join(in_path,in_file)            
            df = pd.read_csv(in_csv
                             , parse_dates=[['date','time']]
                             , infer_datetime_format=True
                             , index_col='date_time')
            # resample to freq. 10Min  
            df = df.resample('10Min', label='right').mean()
            ts1 = time.time()
            print("Column {} has been resampled in {}".format(str(idx), ts1-ts0))
            # Assemble sensors in df10 dataframe
            if idx == 1: 
                df10 = df.copy()
            else: 
                df10 = pd.concat([df10, df], axis=1)            
        # convert datetime to UNIX Timestamp, as requested by NEM
        df10.index = df10.index.astype(np.int64) // 10**9 
        # write to file
        out_file = 'F3-'+ asset +'.csv' 
        out_csv=os.path.join(out_path,out_file)
        df10.to_csv(out_csv)    
        ta1 = time.time()
        print("Asset {} has been written in {}".format(asset, ta1-ta0))          

In [166]:
def F4_all_in_one_csv():
    '''Concatenate resulting csv files per asset (from F4) into a single csv file
    ready for analysis. Convert index to UNIX Timestamp'''

    t0 = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print("Executing F4: all_in_one_csv at {}".format(t0))
    # FUN.CALL
    out_path = files_location()
    in_path  = out_path # modify if in/out-paths needed differnt    
    # FUN.CALL
    assets, sensors, df_trans = read_translations() 
    t0 = time.time()
    
    for idx, asset in enumerate(assets, start=1):
        in_file = 'F3-'+ asset + '.csv'
        in_csv = os.path.join(in_path,in_file)
        df_temp = pd.read_csv(in_csv, index_col = 'date_time')
        df_temp['asset'] = asset 
        if idx==1:
            df = df_temp.copy()
        else:
            df = df.append(df_temp)         
    # reorder df columns to show asset as col1
    cols = df.columns.tolist()
    df = df[cols[-1:] + cols[:-1]] 
    out_path=in_path
    out_file = 'datasets10Min.csv' 
    out_csv=os.path.join(out_path,out_file)
    df.to_csv(out_csv)    
    t1 = time.time()
    print("File {} has been written in {}".format(out_file, t1-t0))                 
    return df    

In [167]:
def remove_empty_files(out_path):
    '''Removes empty files created by dash.dataframe during dd.to_csv() operations. 
       Note that os.walk is limited to the top directory with del dirs[:] '''
    
    # out_path = (r'.\OUTPUT') #used for testing
    for (dirname, dirs, files) in os.walk(out_path):
        for filename in files:
            csv_file = os.path.join(dirname,filename)
            filesize = os.path.getsize(csv_file)
            if filesize < 50:
                # print(csvfile, filesize) <== used for code testing
                os.remove(csv_file)
        del dirs[:]                            

In [168]:
def files_location():
    '''Defines current working directory and subdir to be used as the output container.
    The code is designed to read the original dataset.csv and translations.cvs files
    from the working directory. The resulting new dataset will be written to the 
    OUTPUT subdir.'''
    
    # Define current working dir., where NEM datasets should be allocated
    cwd = r'C:\Home00Ser\Python\NEM Challenge\datasets'
    os.chdir(cwd)
    # Define results output subdir
    out_path = (r'.\OUTPUT5')
    if not(os.path.isdir(out_path)):
        os.mkdir(out_path)   
    return out_path    

In [169]:
if __name__ == "__main__":
    df = main()

Executing F1_csv_asset at 2017-08-14 23:09:24
File A001 has been written in 1123.6245002746582
File A002 has been written in 1124.7763848304749
File A003 has been written in 1097.8318798542023
File A004 has been written in 1076.4352734088898
File A005 has been written in 1031.2572758197784
Executing F2:csv_sensor at 2017-08-15 00:40:23
File A001-1 has been written in 101.44135308265686
File A001-2 has been written in 99.15114951133728
File A001-3 has been written in 100.98581004142761
File A001-4 has been written in 98.94102025032043
File A001-5 has been written in 101.81367754936218
File A001-6 has been written in 99.61627459526062
File A001-7 has been written in 99.9220278263092
File A001-8 has been written in 96.51457715034485
File A001-9 has been written in 96.23719906806946
File A001-10 has been written in 101.18718361854553
File A001-11 has been written in 101.69189262390137
File A001-12 has been written in 100.61859369277954
File A001-13 has been written in 100.16742396354675
Fi

Column 16 has been resampled in 12.679943799972534
Column 17 has been resampled in 11.426796197891235
Column 18 has been resampled in 12.24553918838501
Column 19 has been resampled in 11.600635528564453
Column 20 has been resampled in 11.680091381072998
Column 21 has been resampled in 11.031874895095825
Column 22 has been resampled in 11.738142013549805
Column 23 has been resampled in 11.541831254959106
Column 24 has been resampled in 11.26200795173645
Column 25 has been resampled in 11.867401123046875
Column 26 has been resampled in 12.925169706344604
Column 27 has been resampled in 11.28580641746521
Column 28 has been resampled in 12.077239990234375
Asset A001 has been written in 338.33054065704346
Column 1 has been resampled in 11.402918815612793
Column 2 has been resampled in 12.69534182548523
Column 3 has been resampled in 10.392206907272339
Column 4 has been resampled in 11.73840618133545
Column 5 has been resampled in 14.298771858215332
Column 6 has been resampled in 11.98092722

In [170]:
df

,asset,WGDCTrafoPhase1Temp,WHDRGroupOilPress,WGDCTrafoPhase3Temp,WTURReactivePower,WROTPitchAngleSP,WNACDirection,WGENPhase2Temp,WYAWPressure,WTURReactivePowerAux,...,WHDRGroupOilTemp,WNACAmbTemp,WCNVCosPhi,WTURPowerAux,WGENGenSpeed,WGENPhase1Temp,WCNVNetVoltage,WNACNacelleTemp,WTURPower,WNACWindDirection
date_time,,,,,,,,,,,,,,,,,,,,,
1430439000,A001,0.541919,0.946188,0.541306,0.464225,0.023942,0.632365,0.678605,0.968385,0.496401,...,0.499482,0.352273,0.987495,-0.006425,0.925665,0.670345,62.760000,0.564973,0.484594,0.034610
1430439600,A001,0.541524,0.947391,0.539759,0.463898,0.023304,0.636720,0.674164,1.006540,0.496439,...,0.500185,0.348559,0.988017,-0.006494,0.929502,0.665521,62.814865,0.562886,0.499054,-0.193071
1430440200,A001,0.541172,0.945406,0.539249,0.463815,0.024484,0.636720,0.671611,1.006919,0.496417,...,0.501745,0.345775,0.988010,-0.006485,0.932675,0.662782,62.831740,0.563298,0.498477,-0.063975
1430440800,A001,0.541923,0.946720,0.539951,0.464394,0.023037,0.636720,0.675750,1.007420,0.496524,...,0.501841,0.343308,0.988437,-0.006270,0.932404,0.666791,62.902396,0.558239,0.500386,-0.217021
1430441400,A001,0.543004,0.946798,0.539871,0.465152,0.022420,0.636720,0.667582,1.007043,0.496735,...,0.502408,0.340779,0.985260,-0.005621,0.902527,0.658676,62.843958,0.571563,0.429755,-0.159086
1430442000,A001,0.542826,0.949098,0.539371,0.464601,0.023623,0.626815,0.667339,0.969363,0.496522,...,0.501199,0.343089,0.989713,-0.006215,0.936153,0.658395,62.718656,0.565492,0.541194,-0.102420
1430442600,A001,0.544050,0.950024,0.541913,0.465144,0.022882,0.632886,0.680786,0.949568,0.496371,...,0.501477,0.342769,0.992758,-0.006579,0.955589,0.672140,62.546469,0.555453,0.622016,-0.000300
1430443200,A001,0.546805,0.944836,0.545194,0.466300,0.024797,0.635254,0.693518,0.944370,0.496279,...,0.501490,0.339114,0.993795,-0.006807,0.957790,0.684996,62.576073,0.549729,0.657989,-0.072638
1430443800,A001,0.549281,0.948360,0.547869,0.466289,0.021422,0.608510,0.696775,1.005361,0.496513,...,0.502600,0.336033,0.991575,-0.006369,0.947210,0.688212,62.535969,0.551172,0.565881,0.184530
